### Написать на PyTorch forward и backward полносвязного слоя без использования autograd

In [67]:
import torch
from torch import nn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [68]:
def loss_func(t, y):
    return np.sum((t-y)**2, keepdims=True).flatten()*0.5

def diff_loss_func(t, y):
    return np.sum(np.abs(y-t), kkepdims=True).flatten()

def sigmoid_fun(z):
    return 1./(1+np.exp(-z))

In [69]:
class layer:
    
    def __init__(self, n_input, n_output, lr=0.1):
        self.size = (n_input, n_output)
        self.lr = lr
        self.w = np.zeros(self.size, dtype=np.float32)
        self.b = np.zeros((1, n_output), dtype=np.float32)
        self._clear()
    
    def __call__(self, x):
        if len(x.shape) == 1:
            x = x.reshape(1, -1)
        self.input = x
        self.activations = sigmoid_fun(x.dot(self.w) + self.b)
        return self.activations
    
    def backward(self, grad):
        self.d_sigma = self.activations * (1 - self.activations)
        self.d_w = self.grad_w(grad)
        self.d_b = self.grad_b(grad)
        return self.grad_x(grad)
    
    def grad_w(self, grad):
        return grad * self.input.T * self.d_sigma
    
    def grad_b(self, grad):
        return grad * self.d_sigma
    
    def grad_x(self, grad):
        return self.w.dot(grad) * self.d_sigma
    
    def step(self):
        self.w -= self.d_w * self.lr
        self.b -= self.d_b * self.lr
        self._clear()
    
    def _clear(self):
        self.input = None
        self.activations = None
        self.d_sigma = None
        self.d_w = None
        self.d_b = None
        
layer0 = layer(6, 2)

### Написать 1-2 адаптивных оптимизатора. Pеализация оптимизаторов RMSprop, Adam

https://medium.com/analytics-vidhya/a-complete-guide-to-adam-and-rmsprop-optimizer-75f4502d83be


In [70]:
class RMSprop:
    
    def __init__(self, model, lr=0.01, 
                 v = 0,
                 b = 0.9, 
                 eps = 10 ** (-8) ):
        self.model = model
        self.a = a
        self.v = v  
        self.b = b
        self.eps = eps
    
    def step(self, grad):
        self.v = self.b * self.v + (1 - self.b) * (grad**2)
        self.model -= (self.lr * grad) / (self.v**0.5 + self.eps)

class Adam:
    
    def __init__(self, model, a = 0.001, 
                 m = 0,
                 v = 0,
                 t = 0,
                 b1 = 0.9, 
                 b2 = 0.999, 
                 eps = 10 ** (-8) ):
        self.model = model
        self.a = a
        self.m = m
        self.v = v
        self.t = t
        self.b1 = b1
        self.b2 = b2
        self.eps = eps
    
    def step(self, grad):
        self.t += 1
        self.m = self.b1 * self.m + (1 - self.b1) * grad
        self.v = self.b2 * self.v + (1 - self.b2) * grad
        m_bias = self.m / (1-self.b1)
        v_bias = self.v / (1-self.b2)
        self.model -= (self.a * m_bias) / (v_bias**0.5 + self.eps)
    

### Решить задачу нахождения корней квадратного уравнения методом градиентного спуска

f(x) = x^2 + 4x + 4

x1 = x2 = -0.5


In [71]:
a, b, c = 1, 4, 4

func = lambda x: (a*x**2 + b*x + c)
gradient = lambda x: (2*a*x + b)

def solver(x_start):
    x = torch.tensor(x_start, dtype=torch.float32)
    grad = gradient(x)
    optim = Adam(x, 0.1)
    for i in range(1000):
        optim.step(grad)
        grad = gradient(optim.model)
    print(optim.model)
    
solver(6)

tensor(-2.0000)
